# Chapter 26

# Face Detection II

1. Face detection
2. Face Tracking
3. Face Recognition

## Face detection


## Face Tracking

In [1]:
import cv2 as cv
import dlib


# def draw_text_info():
#     """Draw text information"""

#     # We set the position to be used for drawing text and the menu info:
#     menu_pos_1 = (10, 20)
#     menu_pos_2 = (10, 40)

#     # Write text:
#     cv.putText(frame, "Use '1' to re-initialize tracking", menu_pos_1, cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255))
#     if tracking_face:
#         cv.putText(frame, "tracking the face", menu_pos_2, cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
#     else:
#         cv.putText(frame, "detecting a face to initialize tracking...", menu_pos_2, cv.FONT_HERSHEY_SIMPLEX, 0.5,
#                     (0, 0, 255))


capture = cv.VideoCapture(0)

detector = dlib.get_frontal_face_detector()

tracker = dlib.correlation_tracker()

tracking_face = False

while True:
    ret, frame = capture.read()

    # draw_text_info()

    if tracking_face is False:
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        rects = detector(gray, 0)

        if len(rects) > 0:
            # Start tracking:
            tracker.start_track(frame, rects[0])
            tracking_face = True

    if tracking_face is True:
        print(tracker.update(frame))
        pos = tracker.get_position()
        cv.rectangle(frame, (int(pos.left()), int(pos.top())), (int(pos.right()), int(pos.bottom())), (0, 255, 0), 3)

    key = 0xFF & cv.waitKey(1)

    if key == ord("1"):
        tracking_face = False

    if key == ord('q'):
        break

    cv.imshow("Face tracking using dlib frontal face detector and correlation filters for tracking", frame)

# clean up
capture.release()
cv.destroyAllWindows()

30.67070737072443
20.468808285083572
20.835533015613983
20.027114632787473
17.73990439195485
19.11933915610871
20.831025691077194
17.955703813333166
20.865551294229633
19.37271654267364
17.38809735008587
17.999852109859916
16.88828925390698
19.44132789118658
15.385151060232685
16.557827377904122
14.580826101587064
15.408931636542968
14.667530846817456
16.654007471212097
17.62926928959363
15.302045104806382
15.486457517970752
16.12335770588667
16.15826975549043
16.68014501787084
17.034856071179153
17.968851341721905
19.33925427297836
15.827511654580892
19.286167633467507
20.139061850116168
17.533835698067612
19.443088059386547
19.092843096243875
20.475299901092246
21.14649925668078
19.469195254888024
20.488736996494136
18.169223656080245
19.114138390144838
21.083083109288854
20.535960114242247
20.001494327645965
19.979874388381596
21.164863399500117
17.92303496614852
21.4388798966499
17.907412744932152
20.971468068613394
19.21864889848447
20.88947973085009
21.80601837067179
20.142025607

## Object Tracking

In [3]:
import cv2 as cv
import dlib


# def draw_text_info():
#     """Draw text information"""

#     # We set the position to be used for drawing text and the menu info:
#     menu_pos = (10, 20)
#     menu_pos_2 = (10, 40)
#     menu_pos_3 = (10, 60)
#     info_1 = "Use left click of the mouse to select the object to track"
#     info_2 = "Use '1' to start tracking, '2' to reset tracking and 'q' to exit"

#     # Write text:
#     cv.putText(frame, info_1, menu_pos, cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255))
#     cv.putText(frame, info_2, menu_pos_2, cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255))
#     if tracking_state:
#         cv.putText(frame, "tracking", menu_pos_3, cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
#     else:
#         cv.putText(frame, "not tracking", menu_pos_3, cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))


points = []


def mouse_event_handler(event, x, y, flags, param):
    global points

    if event == cv.EVENT_LBUTTONDOWN:
        points = [(x, y)]

    elif event == cv.EVENT_LBUTTONUP:
        points.append((x, y))


capture = cv.VideoCapture(0)

window_name = "Object tracking using dlib correlation filter algorithm"

cv.namedWindow(window_name)

cv.setMouseCallback(window_name, mouse_event_handler)

tracker = dlib.correlation_tracker()

tracking_state = False

while True:
    ret, frame = capture.read()

    # draw_text_info()

    if len(points) == 2:
        cv.rectangle(frame, points[0], points[1], (0, 0, 255), 3)
        dlib_rectangle = dlib.rectangle(points[0][0], points[0][1], points[1][0], points[1][1])

    if tracking_state == True:
        tracker.update(frame)
        pos = tracker.get_position()
        cv.rectangle(frame, (int(pos.left()), int(pos.top())), (int(pos.right()), int(pos.bottom())), (0, 255, 0), 3)

    key = 0xFF & cv.waitKey(1)

    if key == ord("1"):
        if len(points) == 2:
            tracker.start_track(frame, dlib_rectangle)
            tracking_state = True
            points = []

    if key == ord("2"):
        points = []
        tracking_state = False

    if key == ord('q'):
        break

    cv.imshow(window_name, frame)

# clean up
capture.release()
cv.destroyAllWindows()

## Face Recognition

### dlib

In [ ]:
import cv2 as cv
import dlib
import numpy as np

pose_predictor_5_point = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
detector = dlib.get_frontal_face_detector()


def compare_faces_ordered(encodings, face_names, encoding_to_check):

    distances = list(np.linalg.norm(encodings - encoding_to_check, axis=1))
    return zip(*sorted(zip(distances, face_names)))


def compare_faces(encodings, encoding_to_check):

    return list(np.linalg.norm(encodings - encoding_to_check, axis=1))


def face_encodings(face_image, number_of_times_to_upsample=1, num_jitters=1):
    # Detect faces:
    face_locations = detector(face_image, number_of_times_to_upsample)
    # Detected landmarks:
    raw_landmarks = [pose_predictor_5_point(face_image, face_location) for face_location in face_locations]
    # Calculate the face encoding for every detected face using the detected landmarks for each one:
    return [np.array(face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters)) for
            raw_landmark_set in raw_landmarks]


# Load images:
known_image_1 = cv.imread("jared_1.jpg")
known_image_2 = cv.imread("jared_2.jpg")
known_image_3 = cv.imread("jared_3.jpg")
known_image_4 = cv.imread("obama.jpg")
unknown_image = cv.imread("jared_4.jpg")

# Convert image from BGR (OpenCV format) to RGB (dlib format):
known_image_1 = known_image_1[:, :, ::-1]
known_image_2 = known_image_2[:, :, ::-1]
known_image_3 = known_image_3[:, :, ::-1]
known_image_4 = known_image_4[:, :, ::-1]
unknown_image = unknown_image[:, :, ::-1]

# Crate names for each loaded image:
names = ["jared_1.jpg", "jared_2.jpg", "jared_3.jpg", "obama.jpg"]

# Create the encodings:
known_image_1_encoding = face_encodings(known_image_1)[0]
known_image_2_encoding = face_encodings(known_image_2)[0]
known_image_3_encoding = face_encodings(known_image_3)[0]
known_image_4_encoding = face_encodings(known_image_4)[0]
known_encodings = [known_image_1_encoding, known_image_2_encoding, known_image_3_encoding, known_image_4_encoding]
unknown_encoding = face_encodings(unknown_image)[0]

# Compare faces:
computed_distances = compare_faces(known_encodings, unknown_encoding)
computed_distances_ordered, ordered_names = compare_faces_ordered(known_encodings, names, unknown_encoding)

# Print obtained results:
print(computed_distances)
print(computed_distances_ordered)
print(ordered_names)


### face_dection

In [ ]:

import face_recognition

# Load known images (remember that these images are loaded in RGB order):
known_image_1 = face_recognition.load_image_file("jared_1.jpg")
known_image_2 = face_recognition.load_image_file("jared_2.jpg")
known_image_3 = face_recognition.load_image_file("jared_3.jpg")
known_image_4 = face_recognition.load_image_file("obama.jpg")

# Crate names for each loaded image:
names = ["jared_1.jpg", "jared_2.jpg", "jared_3.jpg", "obama.jpg"]

# Load unknown image (this image is going to be compared against all the previous loaded images):
unknown_image = face_recognition.load_image_file("jared_4.jpg")

# Calculate the encodings for every of the images:
known_image_1_encoding = face_recognition.face_encodings(known_image_1)[0]
known_image_2_encoding = face_recognition.face_encodings(known_image_2)[0]
known_image_3_encoding = face_recognition.face_encodings(known_image_3)[0]
known_image_4_encoding = face_recognition.face_encodings(known_image_4)[0]
known_encodings = [known_image_1_encoding, known_image_2_encoding, known_image_3_encoding, known_image_4_encoding]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]

# Compare the faces:
results = face_recognition.compare_faces(known_encodings, unknown_encoding)

# Print the results:
print(results)